# Homework of Online Convex Optimization

Joseph Vilamarest and Pierre Gaillard

The objective of this homework is to implement from scratch several algorithms from the class to predict daily electricity load in France.

Consultez le repository GitHub à l'aide de ce lien : [TP_OCO](https://github.com/Lilyakhelid/OCO)

In [73]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import plotly.express as px 
from prophet import Prophet
import pandas as pd
import numpy as np

# Historical data used to train the models
data_train = pd.read_csv("data/data_train.csv") 
n0 = len(data_train)

# Test data to be predicted day by day by the model
data_test = pd.read_csv("data/data_test.csv")  
n1 = len(data_test)

data_train.head()

,Unnamed: 0,Date,Load,Load.1,Load.7,Temp,Temp_s95,Temp_s99,Temp_s95_min,Temp_s95_max,...,toy,WeekDays,BH,Year,Month,DLS,Summer_break,Christmas_break,GovernmentResponseIndex,Time
0,1,2012-01-01,51491.500000,51491.500000,51491.500000,11.469339,11.531273,11.398842,11.127697,12.162831,...,0.001338,Sunday,1,2012,1,1,0,20,0.0,15340
1,2,2012-01-02,60683.645833,51491.500000,60683.645833,8.136864,9.514619,10.817107,7.712891,11.110701,...,0.004070,Monday,0,2012,1,1,0,20,0.0,15341
2,3,2012-01-03,67762.104167,60683.645833,67762.104167,8.538243,8.038437,9.633405,6.949912,9.187603,...,0.006803,Tuesday,0,2012,1,1,0,0,0.0,15342
3,4,2012-01-04,68029.229167,67762.104167,68029.229167,8.251058,8.510991,9.266142,8.099900,8.932965,...,0.009535,Wednesday,0,2012,1,1,0,0,0.0,15343
4,5,2012-01-05,69157.395833,68029.229167,69157.395833,9.278617,8.869048,9.056359,8.205341,9.631501,...,0.012267,Thursday,0,2012,1,1,0,0,0.0,15344


In [74]:
# Convert Date column to datetime if not already
data_train['Date'] = pd.to_datetime(data_train['Date'])
data_test['Date'] = pd.to_datetime(data_test['Date'])

The goal is to use combine several models that were trained during the train period (2012-2019) before combining them online after Jan 1st 2020 to perform as well as the best combination.


## Load (or train) expert forecasts

Utilize historical data to train machine learning or statistical models (like GAMs, random forests, gradient boosting,...) to generate predictions for the test period.

Here, we load the forecasts directly.

In [75]:
targets = data_test['Load']
experts = pd.read_csv("data/experts_all.csv",index_col=0)

#--ajout normalisation de la target et des experts--
scaler = MinMaxScaler()
targets_norm = scaler.fit_transform(data_test[['Load']])
experts_norm = pd.DataFrame(
    scaler.fit_transform(experts),
    columns=experts.columns
)
#---------------------------------------------------
T, d = np.shape(experts)


>On fait le choix de normaliser les prédictions des experts en fonction de leur propre étendue (min-max individuel) et non en fonction de l'étendue de la target. L'autre option est aussi envisageable.

In [76]:
experts.head()

,gam,gam_arima,gam_arima2,gam_sa,kal_gam_static,kal_gam_dyn_gs,kal_gam_dyn_em,kal_gam_dyn_big,rf,rf_arima,gb,gb_arima
0,58063.873227,57766.007009,57787.208272,50601.265210,57554.413256,57570.659271,57896.778244,57119.885644,58940.103750,58329.310379,60792.957771,59280.694821
1,57328.504621,57149.385483,56965.657409,51677.091088,57361.824317,57250.828817,56197.987515,52996.859776,57745.006250,56158.407906,57267.954941,54454.935560
2,54508.743389,53689.185505,53529.729479,49274.276630,55001.062314,54529.396481,52674.748739,52973.256081,54341.945208,52312.596375,55164.706663,52528.138069
3,52079.953312,48390.171749,50940.995653,46913.414907,52452.011587,51463.237981,48751.189939,49005.886312,50929.750208,48354.791717,52799.022999,49136.121175
4,51136.507831,46611.321494,49911.200907,46528.176437,51051.319601,49713.903602,47079.744833,47191.339112,52433.911875,49974.822173,51608.467617,47543.796797


---
### Question 1

Plot the data from 2012 to 2021 by showing the electricity load that was used to train the experts, the target to be predicted sequentially and the expert predictions.

In [77]:
all_data = pd.concat([data_train, data_test])
px.line(all_data, x='Date', y='Load', title= 'Data from 2012 to 2021 (train & test) : Electricity load')

In [78]:
all_test = data_test.merge(experts, left_index=True, right_index=True)
variables = ['Load','gam', 'gam_arima', 'gam_arima2', 'gam_sa', 'kal_gam_static',
       'kal_gam_dyn_gs', 'kal_gam_dyn_em', 'kal_gam_dyn_big', 'rf', 'rf_arima',
       'gb', 'gb_arima']
px.line(all_test, x='Date', y=variables, title= 'Load et ses prédictions sur test')

---
## Aggregate the experts online

The goal is to combine the experts with daily updates on the test set, to provide one day ahead predictions.


### Question 2
Implement the update rule of the exponentially weighted average algorithm, that take the current weight vector $p_t$, the loss vector of the experts $l_t$ and a learning rate and return the new weight vector.


>L'algorithme **Exponentially Weighted Average (EWA)** est une méthode utilisée dans le cadre de l'optimisation convexe online. Il met à jour les poids des experts en fonction de leurs performances passées mesurées par des pertes. Cet algorithme garantit un regret logarithmique dans de nombreux cas pratiques.




- **Paramètres** :
  - $d$ : Nombre d'experts.
  - $\eta > 0$ : Learning rate.

- **Variables** :
  - $p_t = (p_{t,1}, p_{t,2}, \dots, p_{t,d})$ : Vecteur des poids à l'instant $t$ (distribution de probabilité sur les experts, $\sum_{i=1}^d p_{t,i} = 1$).
  - $l_t = (l_{t,1}, l_{t,2}, \dots, l_{t,d})$ : Vecteur des pertes des experts.

**Mise à jour des poids** :


1. **Calcul des poids exponentiels non normalisés** :
   $$
   w_{t+1, i} = p_{t,i} \cdot \exp(-\eta \cdot l_{t,i}), \quad \forall i \in \{1, \dots, d\}.
   $$

2. **Normalisation pour garantir que $\sum_{i=1}^d p_{t+1,i} = 1$** :
   $$
   p_{t+1, i} = \frac{w_{t+1, i}}{\sum_{j=1}^d w_{t+1, j}}, \quad \forall i \in \{1, \dots, d\}.
   $$

**Formulation finale** :

   $$
   p_{t+1, i} = \frac{p_{t,i} \cdot \exp(-\eta \cdot l_{t,i})}{\sum_{j=1}^d p_{t,j} \cdot \exp(-\eta \cdot l_{t,j})}, \quad \forall i \in \{1, \dots, d\}.
   $$

---

In [79]:
def EWA_update(eta, p, l):
  """
    Update weights using EWA

    Parameters:
    - eta: float, learning rate.
    - p: np.array of shape (M,), current weights in Δ_M (sum(p) = 1).
    - l: np.array of shape (M,), losses incurred by experts.

    Returns:
    - np.array of shape (M,), updated weights in Δ_M.
  """

  weights = p * np.exp(-eta * l)

  p_future = weights / np.sum(weights) #--on normalise sur les poids pour une somme égal a 1-- 


  return p_future

---
### Question 3 - EWA

Use EWA_update to combine the experts to sequentially predict the targets using the squared loss.

In [80]:
def weights_simulation(eta, T, d, experts_norm, targets_norm):
    """
    Simulation using EWA.
    
    Parameters:
    - eta : float, learning rate.
    - T : int, number of days.
    - d : int, umber of experts.
    - experts_norm : pandas.DataFrame, experts advice normalized.
    - targets_norm : np.array, target normalized.
    
    Return :
    - p : np.array, weights (of size (T+1, d)).
    - pred : np.array, weighted prediction (of size T).
    """
    # Initialize the weight matrix
    p = np.zeros((T+1, d))
    p[0, :] = np.full(d, 1/d)  # Poids uniformes au départ

    # Initialize the prediction of the weighted combination
    pred = np.zeros(T)

    for today in range(T):
        tomorrow = today + 1
        experts_today = experts_norm.iloc[today].values 

        #--predicting the weighted combination of the expert advice--
        pred[today] = np.dot(p[today], experts_today)

        #--loss vectors of the experts for today--
        loss_experts = (targets_norm[today] - experts_today)**2

        #--Update the weight--
        p[tomorrow] = EWA_update(eta, p[today], loss_experts)

    return p, pred


In [81]:
# Set a learning rate
eta = 1
p, pred = weights_simulation(eta, T, d, experts_norm, targets_norm)
print(f'MSE de  : {mean_squared_error(targets_norm,pred)}')

MSE de  : 0.0008870469924476596


In [82]:
#--voir les poids--
print(f'voyons les nan : {np.isnan(p).sum()}')
px.line(pd.DataFrame(p), title= 'Évolution des poids des 12 experts')

voyons les nan : 0


---
### Question 4

a) What is the best theoretical value for eta ? 

b) Plot the evolution of the weights of the experts for the theoretical value of eta and a well-chosen value (tuned by hand to minimize the final error).

La meilleure valeur théorique pour $\eta$ est :

$$\eta^\star = \sqrt{\frac{\log(d)}{T}}$$


In [83]:
eta = np.sqrt(np.log(d)/T)

p, pred = weights_simulation(eta, T, d, experts_norm, targets_norm)
print(f'MSE de  : {mean_squared_error(targets_norm,pred)}')
#--voir les poids--
mse_ewa_theo = mean_squared_error(targets_norm,pred)
print(f'voyons les nan : {np.isnan(p).sum()}')
px.line(pd.DataFrame(p), title= 'Évolution des poids des 12 experts')

MSE de  : 0.0011083757359616607
voyons les nan : 0


>la valeur théorique de $\eta^\star$ n'est pas meilleurs que $\eta$ = 1 arbitrairement.

In [84]:
#--testons tout les etas--
eta_values = np.linspace(0, 1, num=1000)

best_mse = 1
for eta in eta_values:
    p, pred = weights_simulation(eta, T, d, experts_norm, targets_norm)
    mse_temp = mean_squared_error(targets_norm,pred)
    if mse_temp < best_mse:
        best_mse = mse_temp
        best_eta = eta
    else :
        continue

print(f'MSE de  : {best_mse}')
print(f'eta a  : {best_eta}')
#--voir les poids--
mse_ewa = best_mse
print(f'voyons les nan : {np.isnan(p).sum()}')
px.line(pd.DataFrame(p), title= 'Évolution des poids des 12 experts EWA')

MSE de  : 0.0008870469924476596
eta a  : 1.0
voyons les nan : 0


> $\eta^\star$ = 1 minimise le `mse`

---
### Question 5 -- EG

Repeat questions 3 and 4 by using the gradient of the loss (i.e., by using EG) instead of the loss vectors of the experts directly. 

#### **Différence entre EWA et EG**

**Rappel** : Exponentially Weighted Average (EWA)
- Les poids sont mis à jour en fonction des **pertes directes** :
$$
p_{t+1, i} = \frac{p_{t, i} \cdot \exp(-\eta \cdot l_{t, i})}{\sum_{j=1}^d p_{t, j} \cdot \exp(-\eta \cdot l_{t, j})}
$$

**Exponential Gradient (EG)**
- Les poids sont mis à jour en utilisant le **gradient des pertes** :
$$p_{t+1, i} = \frac{p_{t, i} \cdot \exp(-\eta \cdot g_{t, i})}{\sum_{j=1}^d p_{t, j} \cdot \exp(-\eta \cdot g_{t, j})}$$

**Différence clé**

- **EWA** : Basé sur les **pertes directes** : $l_{t, i}$.
- **EG** : Basé sur le **gradient des pertes** $ g_{t, i} $.

In [85]:
def EG_update(eta, p, g):
  """
    Update weights using EG

    Parameters:
    - eta: float, learning rate.
    - p: np.array of shape (M,), current weights in Δ_M (sum(p) = 1).
    - g: np.array of shape (M,), gradient losses incurred by experts.

    Returns:
    - np.array of shape (M,), updated weights in Δ_M.
  """
  return EWA_update(eta, p, g)


La **perte quadratique** est donnée par :
$$
\text{loss} = (\hat{y}_t - y_t)^2
$$
où :
- $ \hat{y}_t = \sum_{i=1}^d p_{t, i} \cdot y_{t, i} $  prédiction pondérée,
- $ y_t $  vérité terrain,
- $ y_{t, i} $  prédiction de l'expert i.

#### Cherchons le gradient :
1. **Calcul de la prédiction pondérée** :
   $$
   \hat{y}_t = \sum_{i=1}^d p_{t, i} \cdot y_{t, i}
   $$

2. **Erreur entre la prédiction pondérée et la vérité terrain** :
   $$
   e_t = \hat{y}_t - y_t
   $$

3. **Gradient pour chaque expert \( i \)** :
   $$
   g_{t, i} = 2 \cdot (\hat{y}_t - y_t) \cdot (y_{t, i} - \hat{y}_t)
   $$


In [86]:
def weights_simulation(eta, T, d, experts_norm, targets_norm): #on adapte pour EG
    """
    Simulation using EG.
    
    Parameters:
    - eta : float, learning rate.
    - T : int, number of days.
    - d : int, umber of experts.
    - experts_norm : pandas.DataFrame, experts advice normalized.
    - targets_norm : np.array, target normalized.
    
    Return :
    - p : np.array, weights (of size (T+1, d)).
    - pred : np.array, weighted prediction (of size T).
    """
    # Initialize the weight matrix
    p = np.zeros((T+1, d))
    p[0, :] = np.full(d, 1/d)  # Poids uniformes au départ

    # Initialize the prediction of the weighted combination
    pred = np.zeros(T)
    for today in range(T):
        tomorrow = today + 1
        experts_today = experts_norm.iloc[today].values 

        #--predicting the weighted combination of the expert advice--
        pred[today] = np.dot(p[today], experts_today)
        #--gradient de la loss quaratique--
        error = pred[today] - targets_norm[today]  
        gradient_losses = 2 * error * (experts_today - pred[today])  
        p[tomorrow] = EG_update(eta, p[today], gradient_losses) 

    return p, pred 

Pour minimiser le regret, la valeur optimale du taux d'apprentissage $\eta$ est donnée par :
$$
\eta^\star = \sqrt{\frac{\log(d)}{T}} \cdot \frac{1}{G}
$$
où :
- $ d $ est le nombre d'experts,
- $ T $ est le nombre total d'itérations,
- $ G $ est une borne supérieure sur la norme des gradients


In [87]:
G = max([np.linalg.norm(2 * (experts.iloc[day, :].values - data_test['Load'].iloc[day]), ord=2) for day in range(T)])
eta =np.sqrt(np.log(d) / T) * ( 1/G )

p, pred = weights_simulation(eta, T, d, experts_norm, targets_norm)
print(f'MSE de  : {mean_squared_error(targets_norm,pred)}')
#--voir les poids--
mse_eg_theo = mean_squared_error(targets_norm,pred)
print(f'voyons les nan : {np.isnan(p).sum()}')
px.line(pd.DataFrame(p), title= 'Évolution des poids des 12 experts')

MSE de  : 0.00114449886995298
voyons les nan : 0


In [88]:
#--testons tout les etas--
eta_values = np.linspace(0, 1, num=1000)

best_mse = 1
for eta in eta_values:
    p, pred = weights_simulation(eta, T, d, experts_norm, targets_norm)
    mse_temp = mean_squared_error(targets_norm,pred)
    if mse_temp < best_mse:
        best_mse = mse_temp
        best_eta = eta
    else :
        continue

print(f'MSE de  : {best_mse}')
print(f'eta a  : {best_eta}')
#--voir les poids--
mse_eg = best_mse
print(f'voyons les nan : {np.isnan(p).sum()}')
px.line(pd.DataFrame(p), title= 'Évolution des poids des 12 experts EG')

MSE de  : 0.0009440074990331141
eta a  : 1.0
voyons les nan : 0


> $\eta^\star$ = 1 minimise le `mse` à nouveau pour EG



---

### Question 6 - Projection on the unit simplex

Here is a simple algorithm to implement (Source at the end of the document).

Let $x=\big(x(i)\big)\in\mathbb{R}^M$,
- Sort the coordinates of x into $y_1\geq y_2\geq\dots\geq y_M$,
- Find
$$ \rho=\max\Big\{j\in 1,\dots,M:\ y_j-\frac{1}{j}\big(\sum_{r=1}^jy_r-1\big)>0\Big\},$$
- Define $z = \frac{1}{\rho}\big(\sum_{r=1}^jy_r-1\big)$,
- Return, for all $i=1, \dots,M$
$$\big(\Pi_{\Delta_M}(x)\big)(i) = \max\big\{x(i)-z, 0\big\}$$

Define the projection function.



In [89]:
# Projection on the simplex.
def proj_simplex(p):
  """
    Projects a vector p onto the simplex Δ_M:
    Δ_M = {z ∈ R^M | ∑ z_i = 1 and z_i ≥ 0 for all i}.
    
    Parameters:
    p : np.array A vector in R^M.
    Returns:
    np.array The projected vector in the simplex Δ_M.
  """

  y = np.sort(p)[::-1] 
   
  cumulative_sum = 0
  rho = -1 #--pour verifier que on a bien un j qui valide la condition--
  for j in range(len(p)):
    cumulative_sum += y[j]
    if y[j] - (cumulative_sum - 1) / (j + 1) > 0:
      rho = j

  z = (np.sum(y[:rho + 1]) - 1) / (rho + 1)
    
  return np.maximum(p - z, 0)


#### Différence entre EG et OGD

- **EG** : Mise à jour multiplicative avec projection implicite :
  $$
  x_{t+1}(i) = \frac{x_t(i) \cdot e^{-\eta \cdot g_t(i)}}{\sum_{j} x_t(j) \cdot e^{-\eta \cdot g_t(j)}}
  $$

- **OGD** : Mise à jour additive avec projection explicite sur simplexe:
  $$
  x_{t+1} = \Pi_K(x_t - \eta \cdot \nabla f_t(x_t))
  $$

### Question 7 - OGD
Implement the update rule of the Online Gradient Descent algorithm, that take the current weight vector $p_t$, the gradient $g_t$ and a learning rate and return the new weight vector.

In [90]:
def OGD_update(eta, p, g):
    """
    Performs an Online Gradient Descent (OGD) update with projection onto the simplex.

    Parameters:
    eta : float Learning rate.
    p : np.array of shape (M,) Vector of weights in Δ_M to be updated.
    g : np.array of shape (M,) Gradient of the loss.
    Returns:
    np.array
        Updated vector of weights in Δ_M.
    """

    y = p - eta * g
    p_future = proj_simplex(y)

    return p_future

### Question 8 
Repeat questions 3 and 4 with OGD.

In [91]:
def weights_simulation(eta, T, d, experts_norm, targets_norm): #on adapte pour OGD
    """
    Simulation using EG.
    
    Parameters:
    - eta : float, learning rate.
    - T : int, number of days.
    - d : int, umber of experts.
    - experts_norm : pandas.DataFrame, experts advice normalized.
    - targets_norm : np.array, target normalized.
    
    Return :
    - p : np.array, weights (of size (T+1, d)).
    - pred : np.array, weighted prediction (of size T).
    """
    # Initialize the weight matrix
    p = np.zeros((T+1, d))
    p[0, :] = np.full(d, 1/d)  # Poids uniformes au départ

    # Initialize the prediction of the weighted combination
    pred = np.zeros(T)

    for today in range(T):
        tomorrow = today + 1
        experts_today = experts_norm.iloc[today].values 

        #--predicting the weighted combination of the expert advice--
        pred[today] = np.dot(p[today], experts_today)
        #--gradient de la loss quaratique--
        error = pred[today] - targets_norm[today]  
        gradient_losses = 2 * error * (experts_today - pred[today])  

        p[tomorrow] = OGD_update(eta, p[today], gradient_losses) 

    return p, pred

Pour minimiser le regret, la valeur optimale du taux d'apprentissage $\eta$ dans l'algorithme OGD est donnée par :
$$
\eta^\star = \frac{D}{G \sqrt{T}}
$$
où :
- $ D $ est le diamètre de l'ensemble faisable,
- $ G $ est une borne supérieure sur la norme du gradient ($ \|g_t\| \leq G $),
- $ T $ est le nombre total d'itérations.

In [92]:
G = max([np.linalg.norm(2 * (experts.iloc[day, :].values - data_test['Load'].iloc[day]), ord=2) for day in range(T)])
D = np.sqrt(2)
eta = D / (G * np.sqrt(T))

p, pred = weights_simulation(eta, T, d, experts_norm, targets_norm)
print(f'MSE de  : {mean_squared_error(targets_norm,pred)}')
#--voir les poids--
mse_ogd_theo = mean_squared_error(targets_norm,pred)
print(f'voyons les nan : {np.isnan(p).sum()}')
px.line(pd.DataFrame(p), title= 'Évolution des poids des 12 experts')

MSE de  : 0.001144490140395685
voyons les nan : 0


In [93]:
#--testons tout les etas--
eta_values = np.linspace(0, 1, num=1000)

best_mse = 1
for eta in eta_values:
    p, pred = weights_simulation(eta, T, d, experts_norm, targets_norm)
    mse_temp = mean_squared_error(targets_norm,pred)
    if mse_temp < best_mse:
        best_mse = mse_temp
        best_eta = eta
    else :
        continue

print(f'MSE de  : {best_mse}')
print(f'eta a  : {best_eta}')
#--voir les poids--
mse_ogd = best_mse
print(f'voyons les nan : {np.isnan(p).sum()}')
px.line(pd.DataFrame(p), title= 'Évolution des poids des 12 experts OGD')

MSE de  : 0.0006187743262945302
eta a  : 1.0
voyons les nan : 0


> $\eta^\star$ = 1 minimise le `mse` à nouveau pour OGD

---
### Question 9
Print a table summarizing the average losses suffered by the best expert, the best fixed combination of experts, EWA, EG, and OGD (each for the best learning rate tuned on data and the theoretical value).

Trouvons le meilleurs experts :

In [94]:
data_test

,Unnamed: 0,Date,Load,Load.1,Load.7,Temp,Temp_s95,Temp_s99,Temp_s95_min,Temp_s95_max,...,toy,WeekDays,BH,Year,Month,DLS,Summer_break,Christmas_break,GovernmentResponseIndex,Time
0,1,2020-03-16,56078.041667,50678.562500,62861.187500,9.887104,10.080688,9.678915,9.146226,10.882901,...,0.206267,Monday,0,2020,3,1,0,0,56.11,18337
1,2,2020-03-17,53813.645833,56078.041667,63156.354167,10.741300,10.378974,9.883176,8.760192,12.098323,...,0.209000,Tuesday,0,2020,3,1,0,0,72.50,18338
2,3,2020-03-18,49906.916667,53813.645833,58545.187500,11.398516,11.027085,10.282211,8.723241,13.299439,...,0.211732,Wednesday,0,2020,3,1,0,0,72.50,18339
3,4,2020-03-19,47498.750000,49906.916667,58315.895833,12.398199,11.923323,10.884522,9.526411,14.187808,...,0.214465,Thursday,0,2020,3,1,0,0,72.50,18340
4,5,2020-03-20,46297.916667,47498.750000,59364.229167,11.889675,12.281893,11.486674,10.723409,13.608776,...,0.217197,Friday,0,2020,3,1,0,0,72.50,18341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,302,2021-01-11,80042.708333,72822.875000,75406.208333,0.834608,0.558479,0.739793,-0.475545,1.479498,...,0.028740,Monday,0,2021,1,1,0,0,66.83,18638
302,303,2021-01-12,77483.291667,80042.708333,77873.770833,4.760695,3.193276,1.546568,1.438928,5.394680,...,0.031480,Tuesday,0,2021,1,1,0,0,66.83,18639
303,304,2021-01-13,71667.541667,77483.291667,78129.250000,7.402015,6.515995,3.432805,5.417378,7.643251,...,0.034220,Wednesday,0,2021,1,1,0,0,66.83,18640
304,305,2021-01-14,69392.312500,71667.541667,78954.562500,7.524001,7.593168,5.068679,7.269445,7.951991,...,0.036960,Thursday,0,2021,1,1,0,0,66.83,18641


In [95]:
y_true = targets
mse_best = np.inf
best_expert = experts.columns[0]
for i in experts.columns :
    y_pred = experts[i]
    mse_temp = mean_squared_error(y_true,y_pred)

    if mse_temp < mse_best :
        mse_best = mse_temp
        best_expert = i
print(best_expert)
print(best_mse)

gb_arima
0.0006187743262945302


In [96]:
data = {
    "Méthode": [
        "best expert gb_arima",
        "EWA eta : 1",
        "EWA eta théorique",
        "EG eta : 1",
        "EG eta théorique",
        "OGD eta : 1",
        "OGD eta théorique"
    ],
    "MSE": [
        best_mse,
        mse_ewa,
        mse_ewa_theo,
        mse_eg,
        mse_eg_theo,
        mse_ogd,
        mse_ogd_theo
    ]
}
df = pd.DataFrame(data)
df = df.sort_values(by="MSE")
fig = px.bar(df, x="Méthode", y="MSE", title="Histogramme des MSE", text="MSE")
fig.show()

### Question 10 (optional)
Test better solutions to tune the learning rate in practice. Ideas: use $\eta_t = \sqrt{1/\sum_{s=1}^t \|g_s\|^2}$ for OGD, where $g_s$ is the gradient at time $s$, or define a grid of possible learning rate and use at time $t$ the learning rate that performed the best until round $t-1$.


>Nous avons testé 1000 valeurs de $\eta$ comprises entre 0 et 1 pour chaque algorithme, dans le but de trouver une meilleure valeur de $\eta$ que 1. Cependant, nous avons constaté que $\eta = 1$ reste la meilleure valeur.

$\eta_t = \sqrt{1/\sum_{s=1}^t \|g_s\|^2}$ for OGD

In [97]:
best_eta = None
best_mse = np.inf
gradient_norms =[]
for t in range(T):
    g_t = 2 * (experts.iloc[t, :].values - data_test['Load'].iloc[t])
    gradient_norms.append(np.linalg.norm(g_t, ord=2))  
    eta_t = np.sqrt(1 / sum(gradient_norms))  # Calcul de eta_t

    p, pred = weights_simulation(eta_t, T, d, experts_norm, targets_norm)
    mse_temp = mean_squared_error(targets_norm, pred)

    if mse_temp < best_mse:
        best_mse = mse_temp
        best_eta = eta_t

print(f"Meilleur eta_t : {best_eta}")
print(f"MSE correspondant : {best_mse}")

Meilleur eta_t : 0.007176400540506915
MSE correspondant : 0.001118463574616676


Different de 1 cette fois ci

---
### Question 11 (optional)
Repeat questions 3 and 4 with ONS or AdaGrad. 

In [106]:
def Adagrad_update(eta, p, g, G_accum):
    """
    Updates weights using Adagrad.

    Parameters:
    - eta: float, learning rate.
    - p: np.array of shape (M,), current weights (sum(p) = 1).
    - g: np.array of shape (M,), gradient of losses.
    - G_accum: np.array of shape (M,), accumulated squared gradients.

    Returns:
    - Updated weights (np.array of shape (M,)).
    - Updated accumulated gradients (np.array of shape (M,)).
    """
    G_accum += g ** 2
    updated_p = p * np.exp(-eta * g / np.sqrt(G_accum + 1e-8))
    return updated_p / np.sum(updated_p), G_accum


def weights_simulation(eta, T, d, experts_norm, targets_norm):
    """
    Runs a simulation with Adagrad.

    Parameters:
    - eta: float, learning rate.
    - T: int, number of time steps (e.g., days).
    - d: int, number of experts.
    - experts_norm: pandas.DataFrame, normalized experts' advice.
    - targets_norm: np.array, normalized targets.

    Returns:
    - p: np.array, weights over time (shape (T+1, d)).
    - pred: np.array, weighted predictions (shape (T,)).
    """
    # Initialize weights
    p = np.zeros((T + 1, d))
    p[0, :] = np.full(d, 1 / d)  # Uniform weights at start

    # Initialize predictions and accumulated gradients
    pred = np.zeros(T)
    G_accum = np.zeros(d)

    for today in range(T):
        tomorrow = today + 1
        experts_today = experts_norm.iloc[today].values

        # Make a prediction
        pred[today] = np.dot(p[today], experts_today)

        # Compute quadratic loss gradient
        error = pred[today] - targets_norm[today]
        gradient_losses = 2 * error * (experts_today - pred[today])

        # Update weights
        p[tomorrow], G_accum = Adagrad_update(eta, p[today], gradient_losses, G_accum)

    return p, pred
#--testons tout les etas--
eta_values = np.linspace(0, 1, num=1000)

best_mse = 1
for eta in eta_values:
    p, pred = weights_simulation(eta, T, d, experts_norm, targets_norm)
    mse_temp = mean_squared_error(targets_norm,pred)
    if mse_temp < best_mse:
        best_mse = mse_temp
        best_eta = eta
    else :
        continue

print(f'MSE de  : {best_mse}')
print(f'eta a  : {best_eta}')
#--voir les poids--
mse_eg = best_mse
print(f'voyons les nan : {np.isnan(p).sum()}')
px.line(pd.DataFrame(p), title= 'Évolution des poids des 12 experts AdaGrad')

MSE de  : 0.0004893861155612305
eta a  : 1.0
voyons les nan : 0


---





### Question 12 (optional)
Build your own forecasting models using data_train based on machine learning models or statistical methods and combine them to see if you improve their performance.


#### Prophet

Prophet est une bibliothèque de prévision de séries temporelles développée par Facebook. Elle est particulièrement adaptée aux séries avec des tendances non linéaires, des variations saisonnières et des événements exceptionnels. 

#### Pourquoi utiliser Prophet ?
- **Tendance :** Modélise les changements à long terme dans les données.
- **Saisonnalité :** Prend en compte les cycles récurrents (hebdomadaires, annuels, etc.).
- **Facilité d'utilisation :** Simple à configurer, même avec des données complexes ou irrégulières.
- **Robustesse :** Gère efficacement les valeurs manquantes et les anomalies.

Prophet est idéal pour des prévisions rapides et fiables, que ce soit pour des données commerciales, climatiques ou autres applications de séries temporelles.


In [98]:
# --training--
vars = ['Date', 'Load']
train_data = data_train[vars].reset_index(drop=True)  
train_data.rename(columns={'Date': 'ds', 'Load': 'y'}, inplace=True)

model_prophet = Prophet()
model_prophet.add_seasonality(name='annual', period=365, fourier_order=10)

model_prophet.fit(train_data)

future1 = model_prophet.make_future_dataframe(periods=0)
forecast1 = model_prophet.predict(future1)
forecast1[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
data_train['prophet forecast'] = forecast1['yhat'].values

# --testing--
test_data = data_test[['Date']].reset_index(drop=True)  
test_data.rename(columns={'Date': 'ds'}, inplace=True) 

forecast_test = model_prophet.predict(test_data)

y_test_pred = forecast_test['yhat']
data_test['prophet forecast'] = y_test_pred.values

22:43:54 - cmdstanpy - INFO - Chain [1] start processing
22:43:54 - cmdstanpy - INFO - Chain [1] done processing


In [103]:
fig = px.line(data_test, x='Date', y=['Load', 'prophet forecast'],title="Predictions du test")
fig.show()

# References
- Duchi, J., Shalev-Shwartz, S., Singer, Y., & Chandra, T. (2008, July). Efficient projections onto the l 1-ball for learning in high dimensions. In Proceedings of the 25th international conference on Machine learning (pp. 272-279).